## 1 Interfacing Between pandas and Model Code

A common workflow for model development is to use pandas for data loading and cleaning before switching over to a modeling library to build the model itself.

The point of contact between pandas and other analysis libraries is usually NumPy arrays. To turn a DataFrame into a NumPy array, use the `values` property

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
data = pd.DataFrame({'x0': range(1, 6),
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                     'y': [-1.5, 0., 3.6, 1.3, -2.]})

In [4]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [5]:
data.columns

Index([u'x0', u'x1', u'y'], dtype='object')

In [6]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

To convert back to a DataFrame, you can pass a two-dimensional ndarray with optional colum names

In [7]:
df2 = pd.DataFrame(data.values, columns=['one', 'two', 'trhee'])

In [8]:
df2

,one,two,trhee
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


The `.values` attribute is intended to be used when your data is homogeneous. If the data is heterogeneous, the result will be an ndarray of Python objects

In [9]:
df3 = data.copy()

In [10]:
df3['strings'] = ['a', 'b', 'c', 'd', 'e']

In [11]:
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [12]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

Using `loc` to indexing with values

In [14]:
data.loc[:, ['x0', 'x1']].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

Prepare "metadata management" manually. Handling a dataset with a non-numeric column

In [18]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'])

In [19]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


Replace the 'category' column with dummy variables

In [20]:
dummies = pd.get_dummies(data.category, prefix='category')

In [21]:
data_with_dummies = data.drop('category', axis=1).join(dummies)

In [22]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


## 2 Creating Model Descriptions with Patsy

***Patsy*** is a Python library for describing statistical models (especially linear models) with a small string-based "formula syntax", which is well supported for specifying linear models in statsmodels.

Patsy's formulas are a special string syntax that looks like:
```
y ~ x0 + x1
```

The syntax a + b is term in the ***design matrix*** created for the model. The `patsy.dmatrices` function take a formula string along with a dataset (DataFrame of or a dict of arrays) and produces design matrices for a linear model

In [5]:
data = pd.DataFrame({'x0': range(1, 6),
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                     'y': [-1.5, 0., 3.6, 1.3, -2.]})

In [6]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
import patsy

In [7]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [8]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [9]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

These Patsy DesignMatrix instances are NumPy ndarrays with additional metadata

In [10]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [11]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

The intercept is a convention for linear models like ordinary least squares(OLS) regression. It can be suppressed by adding the term + 0 to the model

In [14]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

The Patsy objects can be passed directly into algorithms like `numpy.linalg.lstsq`, which performs an ordinary least squares regression

In [15]:
coef, resid, _, _ = np.linalg.lstsq(X, y)

The model metadata is retained in the `design_info` attribute, so you can reattach the model column names to the fitted coefficients to obtain a Series

In [16]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [17]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)

In [18]:
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### 2.1 Data Transformations in Patsy Formulas

You can mix Python code into your Patsy formulas; when evaluating the formula the library will try to find the functions you use in the enclosing scope

In [21]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)

In [22]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

Some commonly used variable transformations include standardizing and centering. Patsy has built-in functions for this purpose

In [23]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

In [24]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

When you perfomr a stateful transformation, which is useing statistics like the mean or standard deviation of the original dataset when tranforming a new dataset. For example, you may fit a model on one dataset, then evaluate the model based on another

The `patsy.build_design_matrces` function can apply transformations to new out-of-sample data using the saved information from the original in-sample dataset

In [25]:
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1, 2, 3, 4]
})

In [26]:
new_X = patsy.build_design_matrices([X.design_info], new_data)

In [27]:
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

Because the plus symbol (+) in the context of Patsy formulas does not mean addition, when you want to add columns from a dataset by name, you must wrap them in the special `I` function

In [29]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

In [30]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

### 2.2 Categorical Data and Patsy

When you use non-numeric terms in a Patsy formula, they are converted to dummy variables by default. If there is an intercept, one of the levels will be left out to avoid collinearity

In [31]:
data = pd.DataFrame({
     'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
     'key2': [0, 1, 0, 1, 0, 1, 0, 0],
     'v1':[1,2,3,4,5,6,7,8],
     'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})

In [32]:
y, X = patsy.dmatrices('v2 ~ key1', data)

In [33]:
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

If you omit the intercept from the model, then columns for each category value will be included in the model design matrix

In [34]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

In [35]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

Numeric columns can be interpreted as categorical with the `C` function

In [36]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

In [37]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

When using multiple categorical terms in a model, you can include interaction terms of the form `key1:key2`, which can be used, for example, in ANOVA models 

In [38]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})

In [39]:
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [40]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)

In [41]:
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [42]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

In [43]:
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

## 3 Introduction to statsmodels

Statsmodels contains more classical frequentist statistical methods, while Bayesian methods and machine learning medels are found in other libraries, including:
- Linear models, generalized linear models, and robust linear models
- linear moxed effects models
- Analysis of variance (ANOVA) methods
- Time Series processes and state space models
- Gneralized method of moments

### 3.1 Estimating Linear Models

Linear models in statsmodels have two different main interfaces: array-based and formula-based

In [44]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

Generate a linear model from some random data

In [52]:
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size
    return mean + np.sqrt(variance) * np.random.randn(size)

In [53]:
np.random.seed(12345)

In [54]:
N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

In [55]:
y = np.dot(X, beta) + eps

In [56]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [57]:
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

The sm.add_constant function can add an intercept column to an existing matrix

In [58]:
X_model = sm.add_constant(X)

In [61]:
X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

The sm.OLS class can fit an ordinary least squares linear regression

In [62]:
model = sm.OLS(y, X)

The model's `fit` method returns a regression result object containing estimated model parameters and other diagnostics

In [63]:
result = model.fit()

In [65]:
result.params

array([ 0.17826108,  0.22303962,  0.50095093])

The `summary` method on result can print a model detailing diagnostic output of the model

In [66]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.413
Method:                 Least Squares   F-statistic:                     24.42
Date:                Sun, 19 Nov 2017   Prob (F-statistic):           7.44e-12
Time:                        16:55:56   Log-Likelihood:                -34.305
No. Observations:                 100   AIC:                             74.61
Df Residuals:                      97   BIC:                             82.42
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1783      0.053      3.364      0.0

Suppose instead that all of the model parameters are in a DataFrame

In [67]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])

In [68]:
data['y'] = y

In [69]:
data.head()

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


Now we can use the statsmodels formula API and Patsy formula strings

In [70]:
results = smf.ols('y ~ col0 + col1 + col2', data).fit()

In [71]:
results.params

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64

In [72]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64

Given new out-of-sample data, you can compute predicted values given the estimated model parameters

In [74]:
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

### 3.2 Estimating Time Series Processes

Simulate some time series data with an autoregressive structure and noise

To be added

## 4 Introduction to scikit-learn

To be added